In [3]:
## import libraries 
import pandas as pd 
import warnings
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, LearningCurveDisplay, ValidationCurveDisplay
from sklearn.preprocessing import StandardScaler
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report, confusion_matrix, RocCurveDisplay
warnings.filterwarnings("ignore")

In [4]:
## importing data as dataframe 
dataset = pd.read_csv('ext_3k_SS01.csv', delimiter=',')
dataset.head()

FileNotFoundError: [Errno 2] No such file or directory: 'ext_3k_SS01.csv'

In [ ]:
data_set = dataset.iloc[:,1:]
data_set.hist()
plt.tight_layout()

In [ ]:
# Transforming geochemical variables 
dt_geochem = data_set.iloc[:,6:21]
dt_rest = data_set.iloc[:,:6]
gc_log = dt_geochem.apply(np.log1p)
gc_log.hist()
plt.tight_layout()

In [ ]:
# Transformed data clubbing 
dt_tfm = pd.concat([dt_rest, gc_log], axis=1)
dt_tfm['Label'] = data_set['Label']
dt_tfm.info()

In [ ]:
## Train test split 
## checking the split no much imbalance 
X = dt_tfm.drop(columns='Label')
y = dt_tfm['Label']
xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True, stratify=y)
print('count of xtrain', xtrain.count())
print('count of xtest', xtest.count())
print('value count of ytrain', ytrain.value_counts())
print('value count of ytest', ytest.value_counts())

In [ ]:
## strandard scaling the data 
sc = StandardScaler()
xtrain_sc = sc.fit_transform(xtrain)
xtrain_df = pd.DataFrame(xtrain_sc)
xtrain_df.hist(bins = 30, figsize=(15,10))
plt.tight_layout()

In [ ]:
## Randomforest classification 
score_list = ['accuracy', 'precision', 'recall', 'f1']
skf = StratifiedKFold(n_splits=5,shuffle=True, random_state=42)
clf = lgb.LGBMClassifier(random_state=42)
cv_results = cross_validate(clf,xtrain_sc,ytrain,cv=skf,scoring = score_list,return_train_score=True, verbose=0)
cv_pd_res = pd.DataFrame(cv_results)
LearningCurveDisplay.from_estimator(clf,xtrain_sc,ytrain,cv=skf,shuffle=True,random_state=42)

In [ ]:
cv_pd_res

In [ ]:
## Hyper_parmaeter tuning of the RaRandomForestClassifier
score_list = ['accuracy', 'precision', 'recall', 'f1']
skf = StratifiedKFold(n_splits=5,shuffle=True, random_state=42)
clf2 =lgb.LGBMClassifier(boosting_type='gbdt', n_estimators=500, reg_alpha=12, colsample_bytree=1.0, learning_rate=0.01, class_weight='balanced', random_state=42, verbose = -1)
cv2_results = cross_validate(clf2,xtrain_sc,ytrain,cv=skf,scoring = score_list,return_train_score=True)
cv_pd_res2 = pd.DataFrame(cv2_results)

In [ ]:
LearningCurveDisplay.from_estimator(clf2,xtrain_sc,ytrain,cv=skf,shuffle=True,random_state=42)

In [ ]:
max_D =[0.2, 0.4, 0.8, 1 , 2 ]
ValidationCurveDisplay.from_estimator(clf2,xtrain_sc,ytrain,param_name='colsample_bytree', param_range= max_D )

In [ ]:
# results of the tuned model for random set; Class label noise is hindering ~70% accuracy 
val_mean = cv_pd_res2.mean()
val_std = cv_pd_res2.std()
val_results = pd.DataFrame()
val_results['mean'] = val_mean
val_results['std'] = val_std
val_results

In [ ]:
## Fitting the tuned model for training data and checking its prediction for test set 
clf_fit = clf2.fit(xtrain_sc,ytrain)
xtest_sc = sc.transform(xtest)
test_pred = clf2.predict(xtest_sc)
print(classification_report(y_true=ytest, y_pred=test_pred))
cm = confusion_matrix(y_true=ytest, y_pred=test_pred, labels=[0,1])
sns.heatmap(cm, annot=True, fmt="d", cmap="coolwarm", 
            xticklabels=["Pred 0","Pred 1"], 
            yticklabels=["True 0","True 1"])
plt.xlabel("Predicted label")
plt.ylabel("True label")
plt.title("Confusion Matrix")
plt.tight_layout()
plt.show()

In [ ]:
RocCurveDisplay.from_estimator(clf2,xtrain_sc,ytrain)

In [ ]:
# Get feature importances (default: split/gain depending on LightGBM version)
importances = clf_fit.feature_importances_  # By default: 'split', can be changed via booster API

# Create Series with feature names
feat_imp = pd.Series(importances, index=dt_tfm.columns[:-1])  # exclude label column
feat_imp = feat_imp.sort_values(ascending=True)  # for horizontal bar plot

# Plot
plt.figure(figsize=(8, 6))
feat_imp.plot(kind='barh')
plt.xlabel('Feature Importance')
plt.title('LightGBM Feature Importances')
plt.tight_layout()
plt.show()

In [ ]:
### Fitting model for all data 
X_sc = sc.transform(X)
clf_all = clf2.fit(X_sc,y)
importances_all = clf_all.feature_importances_

# Create Series with feature names
feat_imp = pd.Series(importances_all, index=dt_tfm.columns[:-1])  # exclude label column
feat_imp = feat_imp.sort_values(ascending=True)  # for horizontal bar plot

# Plot
plt.figure(figsize=(8, 6))
feat_imp.plot(kind='barh')
plt.xlabel('Feature Importance')
plt.title('LightGBM Feature Importances')
plt.tight_layout()
plt.show()

In [ ]:
### fitting model predictions for data cube 
bandset = pd.read_csv('data_cube_ext.csv', delimiter=',')
b_set = bandset.iloc[:,1:]
b_geochem = b_set.iloc[:,6:21]
b_rest = b_set.iloc[:,:6]
log_geochem = b_geochem.apply(np.log1p)
# Transformed data clubbing 
b_final = pd.concat([b_rest, log_geochem], axis=1)
b_final.info()

In [ ]:
### Predictions using clf_all
b_sc = sc.transform(b_final)
RF_model = clf_all.predict(b_sc)

In [ ]:
import rasterio
height = 111
width = 94

# 1. Read the base raster’s metadata
with rasterio.open('Base_data_cube_21.tif') as src:
    meta = src.meta.copy()
    height = src.height
    width = src.width

# 2. Suppose `preds` is your 1D int array of length height*width.
#    First, reshape it:
pred_map = RF_model.reshape((height, width))

# 3. Decide on a valid integer nodata value (e.g. -9999).
#    If your pred_map has NaNs (float), convert them to -9999 now:
if np.issubdtype(pred_map.dtype, np.floating):
    mask = np.isnan(pred_map)
    pred_map = pred_map.astype(np.int64)       # cast to integer
    pred_map[mask] = -9999

# 4. Update the metadata:
meta.update({
    'count': 1,
    'dtype': 'int32',   # or 'int32', whichever you prefer
    'nodata': -9999
})

# 5. Write out your new single-band GeoTIFF
with rasterio.open('TS05_M_lgb_3k_predicted_map.tif', 'w', **meta) as dst:
    dst.write(pred_map, 1)

print("Saved 'predicted_map.tif' with nodata=-9999 and dtype=int64.")


In [ ]:
# Predict class probabilities using your classifier
RF = clf_all.predict_proba(b_sc)
RF_prob = RF[:, 1]  # Probability of class 1

# Read base raster metadata
with rasterio.open('Base_data_cube_21.tif') as src:
    meta = src.meta.copy()
    height = src.height
    width = src.width

# Reshape the 1D prediction array to raster shape
pred_map = RF_prob.reshape((height, width))

# Optional: handle NaNs (though not usually present in RF_model)
if np.isnan(pred_map).any():
    pred_map = np.where(np.isnan(pred_map), -9999, pred_map)
    dtype = 'float32'
    nodata_val = -9999
else:
    dtype = 'float32'
    nodata_val = None  # Or keep existing src.meta['nodata']

# Update metadata for a single-band float raster
meta.update({
    'count': 1,
    'dtype': dtype,
    'nodata': nodata_val
})

# Write out the prediction map
with rasterio.open('TS05_M_lgb_one_cls_predicted_map.tif', 'w', **meta) as dst:
    dst.write(pred_map.astype(dtype), 1)

print("Saved 'TS05_M_XG_one_cls_predicted_map.tif'.")


In [ ]:
xtest_sc.shape